In [10]:
#Let p(i,j,n) denote the proposition that is true when the number n is in the cell in the ith row and the jth column. 

#There are 9×9 × 9 = 729 such propositions. 

#In the sample puzzle p(5,1,6) is true, but p(5,j,6) is false for j = 2,3,...9

#For each cell with a given value, assert p(i,j,n), when the cell in row i and column j has the given value. 

#Assert that every row contains every number. ^ i=1 to 9   ^ n=1 to 9   v j=1 to 9    p(i,j,n)

#Assert that every column contains every number. ^ j=1 to 9   ^ n=1 to 9   v i=1 to 9    p(i,j,n)

#Assert that each of the 3 x 3 blocks contain every number. ^ r=0 to 2   ^ s=0 to 2    ^ n=1 to 9    ^ i=1 to 3    v j=1 to 3 p(3r + i, 3s + j, n)

#Assert that no cell contains more than one number. Take the conjunction over all values of n, n’, i, and j,
#where each variable ranges from 1 to 9 and n!=n', of p(i,j,n) -> not(p(i,j,n')).

#To solve a  Sudoku puzzle, we need to find an assignment of truth values to the 729 variables of the form  p(i,j,n) 
#that makes the conjunction of the assertions true. Those variables that are assigned T yield a solution to the puzzle. 

#A truth table can always be used to determine the satisfiability of a compound proposition. 
#But this is too complex even for modern computers for large problems. 

#There has been much work on developing efficient methods for solving satisfiability problems 
#as many practical problems can be translated into satisfiability problems. 
# import pycosat
import sys, getopt, pycosat, time
from pprint import pprint
from sudoku import *
import pandas as pd

In [6]:
def solve_problem(problemset):
    print('Problem:') 
    pprint(problemset)  
    solve(problemset) 
    print('Answer:')
    pprint(problemset)  

def v(i, j, d):
    """
    Return the number of the variable of cell i, j and digit d,
    which is an integer in the range of 1 to 729 (including).
    """
    return 81 * (i - 1) + 9 * (j - 1) + d


def sudoku_clauses():
    """
    Create the (11745) Sudoku clauses, and return them as a list.
    Note that these clauses are *independent* of the particular
    Sudoku puzzle at hand.
    """
    res = []
    # for all cells, ensure that the each cell:
    for i in range(1, 10):
        for j in range(1, 10):
            # denotes (at least) one of the 9 digits (1 clause)
            res.append([v(i, j, d) for d in range(1, 10)])
            # does not denote two different digits at once (36 clauses)
            for d in range(1, 10):
                for dp in range(d + 1, 10):
                    res.append([-v(i, j, d), -v(i, j, dp)])

    def valid(cells):
        # Append 324 clauses, corresponding to 9 cells, to the result.
        # The 9 cells are represented by a list tuples.  The new clauses
        # ensure that the cells contain distinct values.
        for i, xi in enumerate(cells):
            for j, xj in enumerate(cells):
                if i < j:
                    for d in range(1, 10):
                        res.append([-v(xi[0], xi[1], d), -v(xj[0], xj[1], d)])

    # ensure rows and columns have distinct values
    for i in range(1, 10):
        valid([(i, j) for j in range(1, 10)])
        valid([(j, i) for j in range(1, 10)])
    # ensure 3x3 sub-grids "regions" have distinct values
    for i in 1, 4, 7:
        for j in 1, 4 ,7:
            valid([(i + k % 3, j + k // 3) for k in range(9)])

    assert len(res) == 81 * (1 + 36) + 27 * 324
    return res

def solve(grid):
    #solve a Sudoku problem
    clauses = sudoku_clauses()
    for i in range(1, 10):
        for j in range(1, 10):
            d = grid[i - 1][j - 1]
            # For each digit already known, a clause (with one literal). 
            if d:
                clauses.append([v(i, j, d)])

    # Print number SAT clause  
    numclause = len(clauses)
    print ("P CNF " + str(numclause) +"(number of clauses)")

    # solve the SAT problem
#     start = time.time()
    sol = set(pycosat.solve(clauses))
#     end = time.time()
#     print("Time: "+str(end - start))

    def read_cell(i, j):
        # return the digit of cell i, j according to the solution
        for d in range(1, 10):
            if v(i, j, d) in sol:
                return d

    for i in range(1, 10):
        for j in range(1, 10):
            grid[i - 1][j - 1] = read_cell(i, j)

In [16]:
 #list to view sum of fully solved sudoku puzzles
completedPuzzles = []

#list to view times
completedTimes = []

#for loop to run 100 trials
for i in range(100):
    #reset list of Sudoku Puzzles before they are solved again
    sudokuList = copy.deepcopy(copy_sudoku_lists)
    for count, sudopuzzle in enumerate(sudokuList, 1):
        currentPuzzle = np.array([item for sublist in sudopuzzle for item in sublist])
        start = time.perf_counter()
        solve_problem(sudopuzzle)
        end = time.perf_counter()
        final_time = end - start
        completedPuzzles.append(count)
        completedTimes.append(final_time)

Problem:
[[6, 1, 3, 4, 9, 7, 2, 5, 8],
 [7, 4, 8, 5, 2, 6, 1, 9, 3],
 [5, 2, 9, 8, 1, 3, 7, 4, 6],
 [2, 3, 5, 6, 4, 9, 8, 7, 1],
 [8, 9, 6, 3, 7, 1, 4, 2, 5],
 [4, 7, 1, 2, 8, 5, 6, 3, 9],
 [1, 8, 2, 9, 3, 4, 5, 6, 7],
 [3, 6, 7, 1, 5, 2, 9, 8, 4],
 [9, 5, 4, 7, 6, 8, 3, 1, 2]]
P CNF 11826(number of clauses)
Answer:
[[6, 1, 3, 4, 9, 7, 2, 5, 8],
 [7, 4, 8, 5, 2, 6, 1, 9, 3],
 [5, 2, 9, 8, 1, 3, 7, 4, 6],
 [2, 3, 5, 6, 4, 9, 8, 7, 1],
 [8, 9, 6, 3, 7, 1, 4, 2, 5],
 [4, 7, 1, 2, 8, 5, 6, 3, 9],
 [1, 8, 2, 9, 3, 4, 5, 6, 7],
 [3, 6, 7, 1, 5, 2, 9, 8, 4],
 [9, 5, 4, 7, 6, 8, 3, 1, 2]]
Problem:
[[9, 6, 7, 3, 1, 2, 8, 4, 5],
 [4, 1, 2, 7, 8, 5, 6, 3, 9],
 [5, 8, 3, 6, 4, 9, 1, 2, 7],
 [2, 9, 1, 8, 5, 7, 4, 6, 3],
 [3, 7, 5, 4, 2, 6, 9, 1, 8],
 [6, 4, 8, 9, 3, 1, 7, 5, 2],
 [8, 2, 4, 1, 7, 3, 5, 9, 6],
 [7, 3, 6, 5, 9, 4, 2, 8, 1],
 [1, 5, 9, 2, 6, 8, 3, 7, 4]]
P CNF 11826(number of clauses)
Answer:
[[9, 6, 7, 3, 1, 2, 8, 4, 5],
 [4, 1, 2, 7, 8, 5, 6, 3, 9],
 [5, 8, 3, 6, 4, 9, 1, 2, 7],
 [2,

Answer:
[[5, 9, 7, 4, 3, 2, 8, 1, 6],
 [3, 4, 1, 8, 6, 7, 9, 2, 5],
 [2, 8, 6, 5, 9, 1, 7, 3, 4],
 [7, 1, 5, 6, 2, 9, 3, 4, 8],
 [9, 2, 4, 3, 8, 5, 6, 7, 1],
 [8, 6, 3, 1, 7, 4, 2, 5, 9],
 [1, 3, 9, 2, 5, 6, 4, 8, 7],
 [4, 7, 8, 9, 1, 3, 5, 6, 2],
 [6, 5, 2, 7, 4, 8, 1, 9, 3]]
Problem:
[[8, 6, 5, 1, 9, 7, 2, 4, 3],
 [1, 3, 9, 8, 2, 4, 6, 5, 7],
 [7, 4, 2, 5, 6, 3, 1, 9, 8],
 [9, 2, 6, 3, 8, 1, 5, 7, 4],
 [4, 7, 8, 2, 5, 6, 9, 3, 1],
 [3, 5, 1, 7, 4, 9, 8, 6, 2],
 [6, 9, 7, 4, 1, 2, 3, 8, 5],
 [2, 8, 3, 9, 7, 5, 4, 1, 6],
 [5, 1, 4, 6, 3, 8, 7, 2, 9]]
P CNF 11826(number of clauses)
Answer:
[[8, 6, 5, 1, 9, 7, 2, 4, 3],
 [1, 3, 9, 8, 2, 4, 6, 5, 7],
 [7, 4, 2, 5, 6, 3, 1, 9, 8],
 [9, 2, 6, 3, 8, 1, 5, 7, 4],
 [4, 7, 8, 2, 5, 6, 9, 3, 1],
 [3, 5, 1, 7, 4, 9, 8, 6, 2],
 [6, 9, 7, 4, 1, 2, 3, 8, 5],
 [2, 8, 3, 9, 7, 5, 4, 1, 6],
 [5, 1, 4, 6, 3, 8, 7, 2, 9]]
Problem:
[[2, 4, 1, 9, 7, 6, 3, 5, 8],
 [9, 6, 8, 3, 2, 5, 4, 7, 1],
 [5, 7, 3, 4, 1, 8, 9, 6, 2],
 [8, 5, 4, 2, 9, 3, 6, 1, 7],
 [6, 

P CNF 11826(number of clauses)
Answer:
[[2, 1, 6, 8, 5, 4, 7, 3, 9],
 [8, 5, 7, 9, 1, 3, 2, 4, 6],
 [9, 4, 3, 7, 6, 2, 1, 5, 8],
 [3, 7, 2, 4, 9, 6, 5, 8, 1],
 [5, 6, 9, 1, 7, 8, 4, 2, 3],
 [4, 8, 1, 3, 2, 5, 9, 6, 7],
 [1, 3, 8, 5, 4, 7, 6, 9, 2],
 [6, 9, 4, 2, 8, 1, 3, 7, 5],
 [7, 2, 5, 6, 3, 9, 8, 1, 4]]
Problem:
[[9, 3, 4, 6, 1, 5, 2, 8, 7],
 [7, 5, 2, 3, 4, 8, 6, 1, 9],
 [8, 1, 6, 9, 7, 2, 5, 4, 3],
 [1, 6, 7, 4, 5, 9, 3, 2, 8],
 [5, 2, 3, 7, 8, 1, 9, 6, 4],
 [4, 8, 9, 2, 6, 3, 7, 5, 1],
 [2, 9, 5, 8, 3, 4, 1, 7, 6],
 [6, 4, 1, 5, 9, 7, 8, 3, 2],
 [3, 7, 8, 1, 2, 6, 4, 9, 5]]
P CNF 11826(number of clauses)
Answer:
[[9, 3, 4, 6, 1, 5, 2, 8, 7],
 [7, 5, 2, 3, 4, 8, 6, 1, 9],
 [8, 1, 6, 9, 7, 2, 5, 4, 3],
 [1, 6, 7, 4, 5, 9, 3, 2, 8],
 [5, 2, 3, 7, 8, 1, 9, 6, 4],
 [4, 8, 9, 2, 6, 3, 7, 5, 1],
 [2, 9, 5, 8, 3, 4, 1, 7, 6],
 [6, 4, 1, 5, 9, 7, 8, 3, 2],
 [3, 7, 8, 1, 2, 6, 4, 9, 5]]
Problem:
[[1, 4, 8, 5, 2, 6, 7, 9, 3],
 [2, 3, 7, 1, 8, 9, 4, 6, 5],
 [6, 5, 9, 4, 7, 3, 1, 8, 2],
 [8,

P CNF 11826(number of clauses)
Answer:
[[1, 3, 4, 5, 2, 6, 7, 9, 8],
 [7, 2, 5, 4, 8, 9, 6, 1, 3],
 [6, 9, 8, 3, 7, 1, 4, 2, 5],
 [4, 7, 3, 2, 5, 8, 1, 6, 9],
 [5, 1, 9, 6, 3, 4, 8, 7, 2],
 [8, 6, 2, 9, 1, 7, 5, 3, 4],
 [3, 8, 7, 1, 9, 5, 2, 4, 6],
 [9, 4, 1, 8, 6, 2, 3, 5, 7],
 [2, 5, 6, 7, 4, 3, 9, 8, 1]]
Problem:
[[1, 5, 3, 4, 9, 2, 7, 8, 6],
 [9, 8, 7, 6, 1, 3, 2, 4, 5],
 [2, 4, 6, 8, 7, 5, 9, 3, 1],
 [3, 2, 1, 5, 6, 9, 8, 7, 4],
 [4, 9, 5, 2, 8, 7, 6, 1, 3],
 [7, 6, 8, 1, 3, 4, 5, 9, 2],
 [6, 3, 2, 9, 4, 8, 1, 5, 7],
 [5, 7, 9, 3, 2, 1, 4, 6, 8],
 [8, 1, 4, 7, 5, 6, 3, 2, 9]]
P CNF 11826(number of clauses)
Answer:
[[1, 5, 3, 4, 9, 2, 7, 8, 6],
 [9, 8, 7, 6, 1, 3, 2, 4, 5],
 [2, 4, 6, 8, 7, 5, 9, 3, 1],
 [3, 2, 1, 5, 6, 9, 8, 7, 4],
 [4, 9, 5, 2, 8, 7, 6, 1, 3],
 [7, 6, 8, 1, 3, 4, 5, 9, 2],
 [6, 3, 2, 9, 4, 8, 1, 5, 7],
 [5, 7, 9, 3, 2, 1, 4, 6, 8],
 [8, 1, 4, 7, 5, 6, 3, 2, 9]]
Problem:
[[8, 1, 7, 6, 5, 4, 3, 2, 9],
 [4, 9, 5, 3, 2, 8, 7, 6, 1],
 [6, 2, 3, 9, 1, 7, 4, 5, 8],
 [1,

P CNF 11826(number of clauses)
Answer:
[[3, 6, 4, 2, 8, 5, 1, 7, 9],
 [5, 7, 9, 6, 3, 1, 2, 8, 4],
 [1, 8, 2, 7, 4, 9, 3, 6, 5],
 [6, 2, 3, 9, 5, 4, 8, 1, 7],
 [4, 1, 5, 8, 6, 7, 9, 2, 3],
 [8, 9, 7, 3, 1, 2, 5, 4, 6],
 [7, 4, 8, 1, 9, 3, 6, 5, 2],
 [9, 5, 1, 4, 2, 6, 7, 3, 8],
 [2, 3, 6, 5, 7, 8, 4, 9, 1]]
Problem:
[[4, 8, 6, 5, 7, 1, 3, 2, 9],
 [5, 2, 9, 4, 3, 8, 1, 7, 6],
 [7, 3, 1, 2, 6, 9, 5, 4, 8],
 [3, 6, 4, 9, 8, 7, 2, 5, 1],
 [9, 1, 2, 6, 4, 5, 8, 3, 7],
 [8, 5, 7, 3, 1, 2, 6, 9, 4],
 [6, 7, 3, 1, 2, 4, 9, 8, 5],
 [1, 9, 8, 7, 5, 3, 4, 6, 2],
 [2, 4, 5, 8, 9, 6, 7, 1, 3]]
P CNF 11826(number of clauses)
Answer:
[[4, 8, 6, 5, 7, 1, 3, 2, 9],
 [5, 2, 9, 4, 3, 8, 1, 7, 6],
 [7, 3, 1, 2, 6, 9, 5, 4, 8],
 [3, 6, 4, 9, 8, 7, 2, 5, 1],
 [9, 1, 2, 6, 4, 5, 8, 3, 7],
 [8, 5, 7, 3, 1, 2, 6, 9, 4],
 [6, 7, 3, 1, 2, 4, 9, 8, 5],
 [1, 9, 8, 7, 5, 3, 4, 6, 2],
 [2, 4, 5, 8, 9, 6, 7, 1, 3]]
Problem:
[[5, 6, 3, 2, 4, 7, 9, 8, 1],
 [1, 7, 2, 5, 9, 8, 3, 6, 4],
 [9, 4, 8, 3, 6, 1, 7, 2, 5],
 [8,

...

P CNF 11826(number of clauses)
Answer:
[[3, 2, 7, 5, 8, 9, 4, 6, 1],
 [6, 5, 4, 3, 2, 1, 8, 7, 9],
 [8, 9, 1, 4, 7, 6, 3, 2, 5],
 [5, 1, 6, 9, 4, 8, 2, 3, 7],
 [2, 4, 9, 7, 5, 3, 6, 1, 8],
 [7, 8, 3, 1, 6, 2, 5, 9, 4],
 [1, 7, 8, 2, 3, 5, 9, 4, 6],
 [4, 3, 5, 6, 9, 7, 1, 8, 2],
 [9, 6, 2, 8, 1, 4, 7, 5, 3]]
Problem:
[[2, 3, 9, 5, 7, 1, 4, 6, 8],
 [1, 8, 4, 6, 9, 3, 7, 2, 5],
 [7, 5, 6, 8, 4, 2, 9, 3, 1],
 [5, 4, 1, 9, 2, 8, 6, 7, 3],
 [3, 2, 7, 1, 6, 5, 8, 4, 9],
 [9, 6, 8, 7, 3, 4, 1, 5, 2],
 [8, 7, 5, 2, 1, 6, 3, 9, 4],
 [6, 1, 3, 4, 5, 9, 2, 8, 7],
 [4, 9, 2, 3, 8, 7, 5, 1, 6]]
P CNF 11826(number of clauses)
Answer:
[[2, 3, 9, 5, 7, 1, 4, 6, 8],
 [1, 8, 4, 6, 9, 3, 7, 2, 5],
 [7, 5, 6, 8, 4, 2, 9, 3, 1],
 [5, 4, 1, 9, 2, 8, 6, 7, 3],
 [3, 2, 7, 1, 6, 5, 8, 4, 9],
 [9, 6, 8, 7, 3, 4, 1, 5, 2],
 [8, 7, 5, 2, 1, 6, 3, 9, 4],
 [6, 1, 3, 4, 5, 9, 2, 8, 7],
 [4, 9, 2, 3, 8, 7, 5, 1, 6]]
Problem:
[[6, 1, 3, 5, 8, 9, 4, 7, 2],
 [9, 8, 4, 2, 3, 7, 6, 1, 5],
 [7, 2, 5, 1, 6, 4, 8, 3, 9],
 [2,

P CNF 11826(number of clauses)
Answer:
[[5, 7, 3, 2, 8, 6, 1, 9, 4],
 [4, 9, 1, 7, 3, 5, 6, 2, 8],
 [8, 6, 2, 9, 1, 4, 5, 3, 7],
 [9, 2, 5, 4, 6, 8, 3, 7, 1],
 [6, 3, 7, 1, 5, 2, 4, 8, 9],
 [1, 8, 4, 3, 9, 7, 2, 5, 6],
 [7, 4, 6, 8, 2, 3, 9, 1, 5],
 [2, 1, 8, 5, 4, 9, 7, 6, 3],
 [3, 5, 9, 6, 7, 1, 8, 4, 2]]
Problem:
[[6, 8, 7, 3, 2, 9, 5, 4, 1],
 [1, 2, 9, 5, 6, 4, 8, 3, 7],
 [3, 5, 4, 7, 1, 8, 9, 6, 2],
 [8, 3, 2, 4, 9, 6, 1, 7, 5],
 [7, 4, 5, 8, 3, 1, 2, 9, 6],
 [9, 1, 6, 2, 7, 5, 4, 8, 3],
 [2, 6, 1, 9, 8, 3, 7, 5, 4],
 [5, 9, 3, 1, 4, 7, 6, 2, 8],
 [4, 7, 8, 6, 5, 2, 3, 1, 9]]
P CNF 11826(number of clauses)
Answer:
[[6, 8, 7, 3, 2, 9, 5, 4, 1],
 [1, 2, 9, 5, 6, 4, 8, 3, 7],
 [3, 5, 4, 7, 1, 8, 9, 6, 2],
 [8, 3, 2, 4, 9, 6, 1, 7, 5],
 [7, 4, 5, 8, 3, 1, 2, 9, 6],
 [9, 1, 6, 2, 7, 5, 4, 8, 3],
 [2, 6, 1, 9, 8, 3, 7, 5, 4],
 [5, 9, 3, 1, 4, 7, 6, 2, 8],
 [4, 7, 8, 6, 5, 2, 3, 1, 9]]
Problem:
[[8, 2, 3, 4, 5, 1, 7, 9, 6],
 [1, 4, 6, 7, 2, 9, 3, 8, 5],
 [9, 5, 7, 3, 6, 8, 4, 1, 2],
 [5,

P CNF 11826(number of clauses)
Answer:
[[9, 1, 5, 6, 2, 8, 3, 4, 7],
 [2, 7, 8, 9, 4, 3, 5, 1, 6],
 [6, 4, 3, 5, 7, 1, 2, 9, 8],
 [7, 3, 9, 8, 6, 2, 4, 5, 1],
 [8, 5, 2, 4, 1, 7, 9, 6, 3],
 [4, 6, 1, 3, 9, 5, 8, 7, 2],
 [1, 8, 7, 2, 5, 9, 6, 3, 4],
 [5, 2, 6, 1, 3, 4, 7, 8, 9],
 [3, 9, 4, 7, 8, 6, 1, 2, 5]]
Problem:
[[6, 8, 4, 9, 3, 7, 1, 2, 5],
 [5, 2, 1, 6, 4, 8, 9, 3, 7],
 [7, 3, 9, 2, 1, 5, 8, 4, 6],
 [8, 9, 6, 3, 7, 4, 2, 5, 1],
 [1, 7, 5, 8, 6, 2, 4, 9, 3],
 [3, 4, 2, 1, 5, 9, 6, 7, 8],
 [4, 1, 3, 7, 9, 6, 5, 8, 2],
 [2, 5, 7, 4, 8, 1, 3, 6, 9],
 [9, 6, 8, 5, 2, 3, 7, 1, 4]]
P CNF 11826(number of clauses)
Answer:
[[6, 8, 4, 9, 3, 7, 1, 2, 5],
 [5, 2, 1, 6, 4, 8, 9, 3, 7],
 [7, 3, 9, 2, 1, 5, 8, 4, 6],
 [8, 9, 6, 3, 7, 4, 2, 5, 1],
 [1, 7, 5, 8, 6, 2, 4, 9, 3],
 [3, 4, 2, 1, 5, 9, 6, 7, 8],
 [4, 1, 3, 7, 9, 6, 5, 8, 2],
 [2, 5, 7, 4, 8, 1, 3, 6, 9],
 [9, 6, 8, 5, 2, 3, 7, 1, 4]]
Problem:
[[5, 6, 8, 4, 7, 9, 3, 1, 2],
 [4, 1, 3, 8, 6, 2, 9, 5, 7],
 [7, 2, 9, 5, 3, 1, 8, 6, 4],
 [6,

P CNF 11826(number of clauses)
Answer:
[[1, 4, 9, 8, 5, 7, 3, 2, 6],
 [7, 2, 3, 1, 6, 9, 4, 8, 5],
 [5, 6, 8, 2, 3, 4, 7, 1, 9],
 [4, 8, 2, 3, 7, 6, 5, 9, 1],
 [6, 9, 1, 4, 8, 5, 2, 7, 3],
 [3, 7, 5, 9, 1, 2, 6, 4, 8],
 [9, 3, 4, 6, 2, 8, 1, 5, 7],
 [2, 5, 6, 7, 9, 1, 8, 3, 4],
 [8, 1, 7, 5, 4, 3, 9, 6, 2]]
Problem:
[[6, 1, 3, 4, 9, 7, 2, 5, 8],
 [7, 4, 8, 5, 2, 6, 1, 9, 3],
 [5, 2, 9, 8, 1, 3, 7, 4, 6],
 [2, 3, 5, 6, 4, 9, 8, 7, 1],
 [8, 9, 6, 3, 7, 1, 4, 2, 5],
 [4, 7, 1, 2, 8, 5, 6, 3, 9],
 [1, 8, 2, 9, 3, 4, 5, 6, 7],
 [3, 6, 7, 1, 5, 2, 9, 8, 4],
 [9, 5, 4, 7, 6, 8, 3, 1, 2]]
P CNF 11826(number of clauses)
Answer:
[[6, 1, 3, 4, 9, 7, 2, 5, 8],
 [7, 4, 8, 5, 2, 6, 1, 9, 3],
 [5, 2, 9, 8, 1, 3, 7, 4, 6],
 [2, 3, 5, 6, 4, 9, 8, 7, 1],
 [8, 9, 6, 3, 7, 1, 4, 2, 5],
 [4, 7, 1, 2, 8, 5, 6, 3, 9],
 [1, 8, 2, 9, 3, 4, 5, 6, 7],
 [3, 6, 7, 1, 5, 2, 9, 8, 4],
 [9, 5, 4, 7, 6, 8, 3, 1, 2]]
Problem:
[[9, 6, 7, 3, 1, 2, 8, 4, 5],
 [4, 1, 2, 7, 8, 5, 6, 3, 9],
 [5, 8, 3, 6, 4, 9, 1, 2, 7],
 [2,

P CNF 11826(number of clauses)
Answer:
[[5, 9, 7, 4, 3, 2, 8, 1, 6],
 [3, 4, 1, 8, 6, 7, 9, 2, 5],
 [2, 8, 6, 5, 9, 1, 7, 3, 4],
 [7, 1, 5, 6, 2, 9, 3, 4, 8],
 [9, 2, 4, 3, 8, 5, 6, 7, 1],
 [8, 6, 3, 1, 7, 4, 2, 5, 9],
 [1, 3, 9, 2, 5, 6, 4, 8, 7],
 [4, 7, 8, 9, 1, 3, 5, 6, 2],
 [6, 5, 2, 7, 4, 8, 1, 9, 3]]
Problem:
[[8, 6, 5, 1, 9, 7, 2, 4, 3],
 [1, 3, 9, 8, 2, 4, 6, 5, 7],
 [7, 4, 2, 5, 6, 3, 1, 9, 8],
 [9, 2, 6, 3, 8, 1, 5, 7, 4],
 [4, 7, 8, 2, 5, 6, 9, 3, 1],
 [3, 5, 1, 7, 4, 9, 8, 6, 2],
 [6, 9, 7, 4, 1, 2, 3, 8, 5],
 [2, 8, 3, 9, 7, 5, 4, 1, 6],
 [5, 1, 4, 6, 3, 8, 7, 2, 9]]
P CNF 11826(number of clauses)
Answer:
[[8, 6, 5, 1, 9, 7, 2, 4, 3],
 [1, 3, 9, 8, 2, 4, 6, 5, 7],
 [7, 4, 2, 5, 6, 3, 1, 9, 8],
 [9, 2, 6, 3, 8, 1, 5, 7, 4],
 [4, 7, 8, 2, 5, 6, 9, 3, 1],
 [3, 5, 1, 7, 4, 9, 8, 6, 2],
 [6, 9, 7, 4, 1, 2, 3, 8, 5],
 [2, 8, 3, 9, 7, 5, 4, 1, 6],
 [5, 1, 4, 6, 3, 8, 7, 2, 9]]
Problem:
[[2, 4, 1, 9, 7, 6, 3, 5, 8],
 [9, 6, 8, 3, 2, 5, 4, 7, 1],
 [5, 7, 3, 4, 1, 8, 9, 6, 2],
 [8,

In [17]:
df = pd.DataFrame(list(zip(completedPuzzles,completedTimes)), columns =['Puzzles', 'Avg_Times (Seconds)'])

In [18]:
duplicates = df.groupby(df.Puzzles.tolist(), as_index=False).size()

In [19]:
df1 = df.groupby('Puzzles', as_index=False).mean()

In [20]:
df1['Count of Solutions Found'] =duplicates['size']

In [21]:
df1.to_csv('./Satisfyability_100Trials.csv', index=False)